In [1]:
# ================= Fold Editor (Contour + Separator) =================
# Classes: BG=0, GLOTTIS=1, RIGHT=2, LEFT=3
# Usage:
#   run_fold_editor(r"path", zoom=200)
#   in the path all data should be placed as pairs "image - mask" with specidic Fehling naming convention (see data set)
# Keys:
#   n = save current as *_mask_corrected.png and go next        (composed: glottis on top, no gaps)
#   m = save current and propagate FOLDS **and separator** to all subsequent frames in the same 100-block
#   p = previous (no save)
#   q = quit
# Mouse:
#   LMB drag + release on LEFT panel  : freehand CLOSED contour (green) → apply polygon correction
#   RMB drag + release on LEFT panel  : separator line (yellow) → reassign fold pixels by side (stores separator)
#   LMB drag + release on RIGHT panel : move folds-complex (RIGHT+LEFT) by translation; separator moves with it; GLOTTIS stays unchanged
# Notes:
#   - Glottis (1) is preserved exactly.
#   - RIGHT panel always shows a COMPOSED view (folds "under" glottis; no gaps).


import os, cv2, numpy as np
from glob import glob
from pathlib import Path

BG, GLOTTIS, RIGHT, LEFT = 0, 1, 2, 3

# ---------- IO ----------
def _read_rgb(p):
    bgr = cv2.imread(p, cv2.IMREAD_COLOR)
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB) if bgr is not None else None

def _read_mask(p):
    m = cv2.imread(p, cv2.IMREAD_UNCHANGED)
    if m is None: return None
    if m.ndim == 3: m = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
    return m.astype(np.uint8)

def _save_mask(p, m):
    Path(os.path.dirname(p)).mkdir(parents=True, exist_ok=True)
    cv2.imwrite(p, m.astype(np.uint8))

# ---------- Helpers ----------
def _overlay(rgb, mask):
    out = rgb.copy()
    out[mask==RIGHT]   = (255,  40,  40)
    out[mask==LEFT ]   = ( 40, 180, 255)
    out[mask==GLOTTIS] = (  0,   0,   0)
    return out

def _line_x_of_y(p1, p2, H):
    (y1,x1),(y2,x2) = p1,p2
    y = np.arange(H, dtype=np.float32)
    dy = (y2 - y1); dx = (x2 - x1)
    a = (dx / dy) if abs(dy) > 1e-6 else 0.0
    return x1 + (y - y1) * a

def _midline_from_glottis(mask):
    g = (mask==GLOTTIS).astype(np.uint8)
    ys, xs = np.where(g>0)
    if len(xs) < 10:
        H, W = mask.shape
        return (0, W//2), (H-1, W//2)
    top = (ys.argmin(), xs[ys.argmin()])
    bot = (ys.argmax(), xs[ys.argmax()])
    return (int(top[0]), int(top[1])), (int(bot[0]), int(bot[1]))  # (y,x)

def _apply_polygon_correction(orig_mask, poly_mask, sep_params=None):
    H,W = orig_mask.shape
    newm = orig_mask.copy()
    outside = (~poly_mask)
    rm = ((newm==RIGHT) | (newm==LEFT)) & outside
    newm[rm] = BG

    if sep_params is None:
        p1, p2 = _midline_from_glottis(orig_mask)
        x_line = _line_x_of_y(p1, p2, H)
    else:
        a,b = sep_params
        y = np.arange(H, dtype=np.float32)
        x_line = a*y + b

    xx = np.tile(np.arange(W, dtype=np.float32), (H,1))
    right_side = (xx >= x_line[:,None])

    fill_region = poly_mask & (newm==BG)
    newm[fill_region & right_side]  = LEFT
    newm[fill_region & ~right_side] = RIGHT

    newm[orig_mask==GLOTTIS] = GLOTTIS
    return newm

def _apply_separator(orig_mask, pts_yx):
    H,W = orig_mask.shape
    y1,x1 = pts_yx[0]; y2,x2 = pts_yx[-1]
    dy = (y2 - y1)
    a = (x2 - x1) / dy if abs(dy) > 1e-6 else 0.0
    b = x1 - a*y1

    y = np.arange(H, dtype=np.float32)
    x_line = a*y + b
    xx = np.tile(np.arange(W, dtype=np.float32), (H,1))
    right_side = (xx >= x_line[:,None])

    newm = orig_mask.copy()
    folds = ((newm==RIGHT) | (newm==LEFT))
    newm[folds & right_side]  = LEFT
    newm[folds & ~right_side] = RIGHT
    newm[orig_mask==GLOTTIS] = GLOTTIS
    return newm, (a,b)

# ---------- NEW: “solid underlay” + composition ----------
def _parse_frame_id(rgb_path):
    stem = Path(rgb_path).stem
    try:
        return int(stem.split('_')[0])
    except Exception:
        return None

def _seq_block_end_id(frame_id):
    if frame_id is None: return None
    start = frame_id - ((frame_id - 1) % 100)
    return start + 99

def _translate_folds_only(mask, dy, dx):
    H, W = mask.shape
    cur = mask.copy()
    g = (cur == GLOTTIS)
    r_bin = (cur == RIGHT).astype(np.uint8)
    l_bin = (cur == LEFT ).astype(np.uint8)
    M = np.float32([[1, 0, float(dx)], [0, 1, float(dy)]])
    r_shift = cv2.warpAffine(r_bin, M, (W, H), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    l_shift = cv2.warpAffine(l_bin, M, (W, H), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    out = cur.copy()
    out[(out == RIGHT) | (out == LEFT)] = BG
    out[(r_shift == 1) & (~g)] = RIGHT
    out[(l_shift == 1) & (~g)] = LEFT
    out[g] = GLOTTIS
    return out

def _solid_underlay_from_folds(raw_mask_with_folds, dst_glottis_mask, sep_params=None):
    """
    Build a completely filled folds underlay:
      - Row-wise union of LEFT/RIGHT → fill [xmin:xmax]
      - Assign LEFT/RIGHT by separator (sep_params) or glottis midline
      - Never draw into glottis
    """
    H, W = raw_mask_with_folds.shape
    out = np.zeros_like(raw_mask_with_folds, dtype=np.uint8)

    g = (dst_glottis_mask == GLOTTIS)
    out[g] = GLOTTIS

    if sep_params is None:
        p1, p2 = _midline_from_glottis(dst_glottis_mask)
        x_line = _line_x_of_y(p1, p2, H)
    else:
        a,b = sep_params
        y = np.arange(H, dtype=np.float32)
        x_line = a*y + b

    folds = (raw_mask_with_folds == RIGHT) | (raw_mask_with_folds == LEFT)

    for y in range(H):
        xs = np.where(folds[y])[0]
        if xs.size == 0:
            continue
        xmin, xmax = int(xs.min()), int(xs.max())
        x_range = np.arange(xmin, xmax + 1)
        right_side = (x_range >= x_line[y])
        cls_arr = np.where(right_side, LEFT, RIGHT).astype(np.uint8)
        not_g = ~g[y, x_range]
        to_write = x_range[not_g]
        if to_write.size:
            out[y, to_write] = cls_arr[not_g]

    return out

def _compose_with_glottis_current(raw_mask_with_folds, sep_params_for_frame, dst_glottis_mask):
    underlay = _solid_underlay_from_folds(raw_mask_with_folds, dst_glottis_mask, sep_params_for_frame)
    underlay[dst_glottis_mask == GLOTTIS] = GLOTTIS
    return underlay

def _compose_for_target_frame(raw_src_mask_with_folds, target_original_mask, sep_params_for_src=None):
    return _compose_with_glottis_current(raw_src_mask_with_folds, sep_params_for_src, target_original_mask)

# ---------- Main App ----------
def run_fold_editor(seq_dir, zoom=300):
    zoom = int(zoom); assert zoom >= 50
    scale = zoom / 100.0

    rgb_paths  = sorted(glob(os.path.join(seq_dir, "*_rgb.png")))
    mask_paths = [p.replace("_rgb.png","_mask.png") for p in rgb_paths]
    assert len(rgb_paths) > 0, f"No *_rgb.png in {seq_dir}"

    rgbs  = [_read_rgb(p) for p in rgb_paths]
    masks = [_read_mask(p) for p in mask_paths]
    H,W,_ = rgbs[0].shape
    Hs, Ws = int(H*scale), int(W*scale)

    frame_ids = [_parse_frame_id(p) for p in rgb_paths]
    id_to_idx = {fid: i for i, fid in enumerate(frame_ids) if fid is not None}

    committed  = [m.copy() for m in masks]   # raw editable masks
    sep_params = [None for _ in masks]       # (a,b) per frame

    idx = 0
    drawing = False
    drawing_btn = None
    points = []

    moving = False
    move_start_xy = None
    preview_mask = None
    # NEW: track separator during move
    sep_move_start = None     # (a,b) at drag start (or midline-derived)
    sep_preview = None        # separator used for preview while dragging

    win = "FoldEditor"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(win, Ws*2, Hs)
    cv2.moveWindow(win, 60, 60)

    def to_canvas_left_xy(xc, yc):
        if 0 <= xc < Ws and 0 <= yc < Hs:
            x = int(xc / scale); y = int(yc / scale)
            return x,y
        return None

    def to_canvas_right_xy(xc, yc):
        xr = xc - Ws
        if 0 <= xr < Ws and 0 <= yc < Hs:
            x = int(xr / scale); y = int(yc / scale)
            return x,y
        return None

    def draw_ui(mask_to_show, sep_for_view):
        rgb = rgbs[idx]
        composed = _compose_with_glottis_current(mask_to_show, sep_for_view, committed[idx])
        L = cv2.resize(rgb, (Ws, Hs), interpolation=cv2.INTER_LINEAR)
        R = cv2.resize(_overlay(rgb, composed), (Ws, Hs), interpolation=cv2.INTER_NEAREST)
        canvas = np.zeros((Hs, Ws*2, 3), dtype=np.uint8)
        canvas[:, :Ws] = cv2.cvtColor(L, cv2.COLOR_RGB2BGR)
        canvas[:, Ws:] = cv2.cvtColor(R, cv2.COLOR_RGB2BGR)

        if drawing and len(points) > 1:
            pts = np.array([[int(x*scale), int(y*scale)] for (y,x) in points], np.int32)
            color = (0,255,0) if drawing_btn=='L' else (0,255,255)
            cv2.polylines(canvas[:, :Ws], [pts], isClosed=False, color=color, thickness=2, lineType=cv2.LINE_AA)

        name = os.path.basename(rgb_paths[idx])
        status = f"{name}   |   LMB: contour(L) / move folds(R)   RMB: separator(L)   |   n=save&next  m=save&propagate  p=prev  q=quit   |   zoom={zoom}%"
        cv2.putText(canvas, status, (10, 24), cv2.FONT_HERSHEY_SIMPLEX, 0.58, (240,240,240), 2, cv2.LINE_AA)
        cv2.putText(canvas, "ORIGINAL", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 1, cv2.LINE_AA)
        cv2.putText(canvas, "MASK OVERLAY (composed)", (Ws+10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 1, cv2.LINE_AA)
        cv2.imshow("FoldEditor", canvas)

    def _ensure_sep(idx_local):
        """Return a valid separator (a,b) for this frame; if none, derive from GL midline."""
        if sep_params[idx_local] is not None:
            return sep_params[idx_local]
        p1, p2 = _midline_from_glottis(committed[idx_local])
        # compute a,b from two points (y,x): x = a*y + b
        (y1,x1),(y2,x2) = p1,p2
        dy = (y2 - y1)
        a = (x2 - x1) / dy if abs(dy) > 1e-6 else 0.0
        b = x1 - a*y1
        return (a,b)

    def on_mouse(event, x, y, flags, param):
        nonlocal drawing, drawing_btn, points
        nonlocal moving, move_start_xy, preview_mask, sep_move_start, sep_preview

        mL = to_canvas_left_xy(x, y)
        mR = to_canvas_right_xy(x, y)

        # ---- LEFT: polygon
        if event == cv2.EVENT_LBUTTONDOWN and mL is not None:
            drawing = True; drawing_btn = 'L'; points = [(mL[1], mL[0])]
        elif event == cv2.EVENT_MOUSEMOVE and drawing and drawing_btn=='L' and mL is not None:
            py, px = mL[1], mL[0]
            if (py,px) != points[-1]: points.append((py,px))
        elif event == cv2.EVENT_LBUTTONUP and drawing and drawing_btn=='L':
            if len(points) >= 3:
                y0,x0 = points[0]; y1,x1 = points[-1]
                if (x1-x0)**2 + (y1-y0)**2 <= 10**2:
                    poly = np.array([[x,y] for (y,x) in points], np.int32)
                    poly_mask = np.zeros(committed[idx].shape, np.uint8)
                    cv2.fillPoly(poly_mask, [poly], 1)
                    committed[idx] = _apply_polygon_correction(
                        committed[idx], poly_mask.astype(bool), sep_params[idx]
                    )
            drawing = False; drawing_btn = None; points = []

        # ---- LEFT: separator (stores sep_params[idx])
        if event == cv2.EVENT_RBUTTONDOWN and mL is not None:
            drawing = True; drawing_btn = 'R'; points = [(mL[1], mL[0])]
        elif event == cv2.EVENT_MOUSEMOVE and drawing and drawing_btn=='R' and mL is not None:
            py, px = mL[1], mL[0]
            if (py,px) != points[-1]: points.append((py,px))
        elif event == cv2.EVENT_RBUTTONUP and drawing and drawing_btn=='R':
            if len(points) >= 2:
                newm, ab = _apply_separator(committed[idx], points)
                committed[idx] = newm
                sep_params[idx] = ab  # save separator
            drawing = False; drawing_btn = None; points = []

        # ---- RIGHT: move folds **and separator**
        if event == cv2.EVENT_LBUTTONDOWN and mR is not None:
            moving = True
            move_start_xy = (mR[0], mR[1])          # (x,y) in original coords
            preview_mask = committed[idx].copy()
            sep_move_start = _ensure_sep(idx)       # capture a,b at drag start
            sep_preview = sep_move_start

        elif event == cv2.EVENT_MOUSEMOVE and moving and mR is not None and move_start_xy is not None:
            dx = int(round(mR[0] - move_start_xy[0]))
            dy = int(round(mR[1] - move_start_xy[1]))
            # preview folds
            preview_mask = _translate_folds_only(committed[idx], dy, dx)
            # preview separator translation: x' = a*y' + (b - a*dy + dx)
            a0, b0 = sep_move_start
            sep_preview = (a0, b0 - a0*dy + dx)

        elif event == cv2.EVENT_LBUTTONUP and moving:
            if mR is not None and move_start_xy is not None:
                dx = int(round(mR[0] - move_start_xy[0]))
                dy = int(round(mR[1] - move_start_xy[1]))
                committed[idx] = _translate_folds_only(committed[idx], dy, dx)
                # commit translated separator
                a0, b0 = sep_move_start
                sep_params[idx] = (a0, b0 - a0*dy + dx)
            moving = False
            move_start_xy = None
            preview_mask = None
            sep_move_start = None
            sep_preview = None

    cv2.setMouseCallback("FoldEditor", on_mouse)

    print("[FoldEditor] Opened. LMB=contour (left) / move folds+divider (right), RMB=separator (left). 'n' saves & next, 'm' saves & propagates (incl. divider), 'p' prev, 'q' quit.")
    while True:
        # Show composed (no gaps). If moving, compose with sep_preview; else sep_params[idx]
        current_for_view = preview_mask if (moving and preview_mask is not None) else committed[idx]
        sep_for_view = sep_preview if (moving and sep_preview is not None) else sep_params[idx]
        draw_ui(current_for_view, sep_for_view)

        k = cv2.waitKey(20) & 0xFF
        if k == 255:
            continue
        if k == ord('q') or k == 27:
            break

        if k == ord('n'):
            # Save composed current (uses current frame's separator if present; else midline)
            ab = sep_params[idx]
            composed_now = _compose_with_glottis_current(committed[idx], ab, committed[idx])
            out_p = mask_paths[idx].replace("_mask.png", "_mask_corrected.png")
            _save_mask(out_p, composed_now)
            print("saved:", out_p)
            if idx == len(rgbs)-1:
                print("[FoldEditor] Reached last frame. All done.")
            idx = min(len(rgbs)-1, idx+1)

        elif k == ord('m'):
            # Save composed current and propagate folds **and the same separator** to rest of 100-block
            cur_id = frame_ids[idx]
            if cur_id is None:
                print("[WARN] Cannot parse frame id; skipping propagate.")
            else:
                src_raw = committed[idx]
                src_sep = _ensure_sep(idx)            # ensure we have (a,b)
                # save current
                out_cur = mask_paths[idx].replace("_mask.png", "_mask_corrected.png")
                out_mask = _compose_with_glottis_current(src_raw, src_sep, committed[idx])
                _save_mask(out_cur, out_mask)
                print("saved:", out_cur, "(source for propagation, divider included)")

                block_end = _seq_block_end_id(cur_id)
                for target_id in range(cur_id + 1, block_end + 1):
                    j = id_to_idx.get(target_id, None)
                    if j is None:
                        continue
                    dst_composed = _compose_for_target_frame(src_raw, masks[j], src_sep)
                    out_p = mask_paths[j].replace("_mask.png", "_mask_corrected.png")
                    _save_mask(out_p, dst_composed)
                    committed[j] = dst_composed
                    # also propagate the SAME separator to target frames
                    sep_params[j] = src_sep
                    print("propagated to:", out_p)

                if idx == len(rgbs)-1:
                    print("[FoldEditor] Reached last frame. Propagation complete.")
                idx = min(len(rgbs)-1, idx+1)

        elif k == ord('p'):
            idx = max(0, idx-1)

    cv2.destroyAllWindows()
    print("[FoldEditor] Closed.")

In [ ]:
seq_dir = r"C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension"
run_fold_editor(seq_dir)

[FoldEditor] Opened. LMB=contour (left) / move folds+divider (right), RMB=separator (left). 'n' saves & next, 'm' saves & propagates (incl. divider), 'p' prev, 'q' quit.
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48501_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48502_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48503_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48504_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48505_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48506_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\48507_mask_corrected.png
saved: C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data

In [1]:
import os

def replace_corrected_masks(DIR: str):
    """
    Goes through the specified directory and processes image–mask triplets named like:
    10001_mask.png, 10001_mask_corrected.png, 10001_rgb.png, etc.

    Function behavior:
    1. Deletes all files ending with '_mask' (e.g., '10001_mask.png').
    2. Renames all corresponding '_mask_corrected' files to '_mask'
       (e.g., '10001_mask_corrected.png' → '10001_mask.png').

    This is typically used after manual correction of segmentation masks,
    where corrected masks should replace the original ones.

    Parameters
    ----------
    DIR : str
        Path to the directory containing the triplets.
    """
    for filename in os.listdir(DIR):
        file_path = os.path.join(DIR, filename)

        # Skip non-files (like folders)
        if not os.path.isfile(file_path):
            continue

        # Remove old (non-corrected) mask files
        if filename.endswith('_mask.png') and not filename.endswith('_mask_corrected.png'):
            os.remove(file_path)
            print(f"Removed: {filename}")

        # Rename corrected mask files to standard name
        elif filename.endswith('_mask_corrected.png'):
            new_name = filename.replace('_mask_corrected.png', '_mask.png')
            new_path = os.path.join(DIR, new_name)
            os.rename(file_path, new_path)
            print(f"Renamed: {filename} → {new_name}")

In [2]:
dir = r"C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension"
replace_corrected_masks(dir)

Removed: 40001_mask.png
Renamed: 40001_mask_corrected.png → 40001_mask.png
Removed: 40002_mask.png
Renamed: 40002_mask_corrected.png → 40002_mask.png
Removed: 40003_mask.png
Renamed: 40003_mask_corrected.png → 40003_mask.png
Removed: 40004_mask.png
Renamed: 40004_mask_corrected.png → 40004_mask.png
Removed: 40005_mask.png
Renamed: 40005_mask_corrected.png → 40005_mask.png
Removed: 40006_mask.png
Renamed: 40006_mask_corrected.png → 40006_mask.png
Removed: 40007_mask.png
Renamed: 40007_mask_corrected.png → 40007_mask.png
Removed: 40008_mask.png
Renamed: 40008_mask_corrected.png → 40008_mask.png
Removed: 40009_mask.png
Renamed: 40009_mask_corrected.png → 40009_mask.png
Removed: 40010_mask.png
Renamed: 40010_mask_corrected.png → 40010_mask.png
Removed: 40011_mask.png
Renamed: 40011_mask_corrected.png → 40011_mask.png
Removed: 40012_mask.png
Renamed: 40012_mask_corrected.png → 40012_mask.png
Removed: 40013_mask.png
Renamed: 40013_mask_corrected.png → 40013_mask.png
Removed: 40014_mask.png
R

In [3]:
import os
import re

def renumber_pairs_in_dir(folder, old_start=3001, new_start=1501,
                          rgb_suffix="_rgb.png", mask_suffix="_mask.png", dry_run=False):
    """
    Renames all paired files in a folder, shifting numeric prefixes.
    Example:
        03001_rgb.png -> 01501_rgb.png
        03001_mask.png -> 01501_mask.png

    Args:
        folder      : directory containing the files
        old_start   : first old number (e.g., 3001)
        new_start   : number to start from (e.g., 1501)
        rgb_suffix  : suffix pattern for RGB files
        mask_suffix : suffix pattern for MASK files
        dry_run     : if True, only prints what would happen (no renaming)
    """
    files = sorted(os.listdir(folder))
    pattern_rgb = re.compile(rf"^(\d+){re.escape(rgb_suffix)}$")
    pattern_mask = re.compile(rf"^(\d+){re.escape(mask_suffix)}$")
    
    # collect all unique numeric prefixes
    ids = set()
    for fname in files:
        m1 = pattern_rgb.match(fname)
        m2 = pattern_mask.match(fname)
        if m1:
            ids.add(int(m1.group(1)))
        elif m2:
            ids.add(int(m2.group(1)))

    if not ids:
        print("⚠️ No matching files found.")
        return
    
    ids = sorted(ids)
    offset = new_start - ids[0]
    print(f"Found {len(ids)} sequences. Renumbering {ids[0]}..{ids[-1]} → starts at {new_start}")
    print(f"Offset = {offset:+d}")

    renamed = 0
    for old_id in ids:
        new_id = old_id + offset
        for suffix in [rgb_suffix, mask_suffix]:
            old_name = f"{old_id:05d}{suffix}"
            new_name = f"{new_id:05d}{suffix}"
            old_path = os.path.join(folder, old_name)
            new_path = os.path.join(folder, new_name)
            if os.path.exists(old_path):
                if dry_run:
                    print(f"[DRY] {old_name} -> {new_name}")
                else:
                    os.rename(old_path, new_path)
                    renamed += 1
    if dry_run:
        print("✅ Dry run complete. Set dry_run=False to apply changes.")
    else:
        print(f"✅ Done. {renamed} files renamed.")

# Example usage:
# renumber_pairs_in_dir(r"D:\data\fehling_dataset\train", old_start=3001, new_start=1501, dry_run=True)


In [4]:
renumber_pairs_in_dir(r"C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\test_val_extension")

Found 2000 sequences. Renumbering 3001..5000 → starts at 1501
Offset = -1500
✅ Done. 4000 files renamed.


In [1]:
# FIX MASKS (CREATE UNIFIED GLOTTAL COMPLEX)

import os
from glob import glob
from PIL import Image
import cv2
import numpy as np
from pathlib import Path

BG, GLOTTIS, RIGHT, LEFT = 0, 1, 2, 3

def _read_rgb(path):
    im_bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    return cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB) if im_bgr is not None else None

def _read_mask(path):
    m = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if m.ndim == 3:
        m = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
    return m.astype(np.uint8)

def _save_mask(path, mask):
    Path(os.path.dirname(path)).mkdir(parents=True, exist_ok=True)
    cv2.imwrite(path, mask.astype(np.uint8))

def _extract_scribbles(rgb):
    """Detect blue (→ RIGHT) and green (→ LEFT) scribble lines with strict tolerance."""
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    # blue (BGR ~ 255,0,0)
    lower_blue  = np.array([240, 0, 0])
    upper_blue  = np.array([255, 20, 20])
    blue_mask = cv2.inRange(bgr, lower_blue, upper_blue)

    # green (BGR ~ 0,255,0)
    lower_green = np.array([0, 240, 0])
    upper_green = np.array([20, 255, 20])
    green_mask = cv2.inRange(bgr, lower_green, upper_green)

    return (blue_mask>0).astype(np.uint8), (green_mask>0).astype(np.uint8)

def _fill_closed_contour(line_mask, min_area=30):
    """
    Given a binary scribble line (closed), return its filled interior.
    """
    if line_mask.sum() == 0:
        return np.zeros_like(line_mask, np.uint8)

    # Find contours and fill them
    cnts, _ = cv2.findContours(line_mask.astype(np.uint8),
                               cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled = np.zeros_like(line_mask, np.uint8)
    for c in cnts:
        if cv2.contourArea(c) >= min_area:
            cv2.drawContours(filled, [c], -1, 1, thickness=cv2.FILLED)
    return filled

def correct_folds_from_closed_contours(root_dir, target_dir):
    """
    Replace folds in *_mask.png with filled regions from closed blue/green scribbles.
    """
    os.makedirs(target_dir, exist_ok=True)
    rgb_paths = sorted(glob(os.path.join(root_dir, "*_rgb.png")))
    for rgb_path in rgb_paths:
        mask_path = rgb_path.replace("_rgb.png", "_mask.png")
        if not os.path.exists(mask_path):
            continue

        rgb   = _read_rgb(rgb_path)
        mask  = _read_mask(mask_path)
        gl    = (mask == GLOTTIS)

        # start fresh (remove folds)
        corrected = mask.copy()
        corrected[(mask == RIGHT) | (mask == LEFT)] = BG

        # scribbles
        blue_line, green_line = _extract_scribbles(rgb)

        # fill interiors of closed contours
        right_fill = _fill_closed_contour(blue_line)
        left_fill  = _fill_closed_contour(green_line)

        # apply them, restricted to non-glottis
        corrected[right_fill.astype(bool) & (~gl)] = RIGHT
        corrected[left_fill.astype(bool)  & (~gl)] = LEFT

        # restore glottis
        corrected[gl] = GLOTTIS

        out_path = os.path.join(
            target_dir,
            os.path.basename(mask_path).replace("_mask.png", "_mask_corrected.png")
        )
        _save_mask(out_path, corrected)

        print(f"✔ {os.path.basename(mask_path)} → {os.path.basename(out_path)} "
              f"(blue_px={int(blue_line.sum())}, green_px={int(green_line.sum())})")

def fix_masks_make_unified_complex(root_dir):
    """
    Walk all '*_mask.png' files under `root_dir` and fix the segmentation so that:
      1) There is exactly ONE unified folds–glottis complex (labels {1,2,3}).
         Any folds (2 or 3) outside this main complex are set to background (0).
         Glottis (1) is NEVER changed.
      2) No background (0) is allowed *inside* the complex:
         all background "holes" enclosed by {1,2,3} are reassigned to the nearest /
         most-adjacent fold (2 or 3). Glottis (1) remains untouched.

    Labels: 0=background, 1=glottis (GT, immutable), 2=right fold, 3=left fold.
    """
    try:
        from scipy.ndimage import (
            label as cc_label,
            binary_fill_holes,
            binary_dilation,
            generate_binary_structure,
            distance_transform_edt,
        )
        _use_scipy = True
    except Exception:
        # Minimal fallbacks using OpenCV if SciPy is unavailable
        import cv2
        _use_scipy = False

        def _cc_label(mask):
            # 8-connected components
            num, lab, stats, _ = cv2.connectedComponentsWithStats(
                mask.astype(np.uint8), connectivity=8
            )
            return lab, num - 1  # lab includes background=0

        def _binary_fill_holes(mask):
            # flood fill from 4 corners to get outside background; holes are the rest of bg
            h, w = mask.shape
            inv = (~mask).astype(np.uint8)  # 1 where background
            ff = inv.copy()
            mm = np.zeros((h + 2, w + 2), np.uint8)
            for seed in [(0, 0), (0, w - 1), (h - 1, 0), (h - 1, w - 1)]:
                cv2.floodFill(ff, mm.copy(), seedPoint=seed, newVal=2)
            outside = (ff == 2)
            holes = (~outside) & (~mask)
            filled = mask | holes
            return filled, holes

        def _binary_dilation(mask):
            k = np.ones((3, 3), np.uint8)
            return cv2.dilate(mask.astype(np.uint8), k, iterations=1).astype(bool)

        def _distance_transform(mask):
            # distance to True pixels
            import cv2
            inv = (~mask).astype(np.uint8)
            dist = cv2.distanceTransform(inv, cv2.DIST_L2, 3)
            return dist

    def _process_single(arr):
        """
        arr: (H,W) uint8 in {0,1,2,3}
        returns corrected arr
        """
        arr = arr.copy()

        glottis = (arr == 1)
        fold2 = (arr == 2)
        fold3 = (arr == 3)
        complex_mask = glottis | fold2 | fold3

        if _use_scipy:
            st = generate_binary_structure(2, 2)  # 8-connectivity
            cc, n = cc_label(complex_mask, st)
        else:
            cc, n = _cc_label(complex_mask)

        # ---- choose the main complex ----
        if n <= 1:
            main = complex_mask
        else:
            if glottis.any():
                # pick the component that overlaps the glottis the most
                ids, counts = np.unique(cc[glottis], return_counts=True)
                ids = ids[ids > 0]
                if len(ids):
                    main_id = ids[np.argmax(counts)]
                else:
                    # fallback: largest by area
                    areas = np.bincount(cc.ravel())
                    areas[0] = 0
                    main_id = np.argmax(areas)
            else:
                areas = np.bincount(cc.ravel())
                areas[0] = 0
                main_id = np.argmax(areas)
            main = (cc == main_id)

        # ---- remove stray folds outside the main complex (glottis never changed) ----
        outside_folds = (~main) & ((arr == 2) | (arr == 3))
        arr[outside_folds] = 0

        # recompute masks after cleanup
        fold2 = (arr == 2)
        fold3 = (arr == 3)
        complex_mask = glottis | fold2 | fold3

        # ---- fill background holes inside the complex (assign to fold 2 or 3) ----
        if _use_scipy:
            st = generate_binary_structure(2, 2)
            filled = binary_fill_holes(complex_mask, structure=st)
            holes = filled & (~complex_mask)  # background islands fully enclosed
        else:
            filled, holes = _binary_fill_holes(complex_mask)

        if holes.any():
            # label hole components
            if _use_scipy:
                h_lbl, h_num = cc_label(holes, st)
            else:
                h_lbl, h_num = _cc_label(holes)

            # Precompute distances for fallback
            if _use_scipy:
                dist2 = distance_transform_edt(~fold2) if fold2.any() else np.full(arr.shape, np.inf, dtype=np.float32)
                dist3 = distance_transform_edt(~fold3) if fold3.any() else np.full(arr.shape, np.inf, dtype=np.float32)
            else:
                dist2 = _distance_transform(fold2) if fold2.any() else np.full(arr.shape, np.inf, dtype=np.float32)
                dist3 = _distance_transform(fold3) if fold3.any() else np.full(arr.shape, np.inf, dtype=np.float32)

            for hid in range(1, h_num + 1):
                hm = (h_lbl == hid)

                # boundary of the hole (8-neighborhood)
                if _use_scipy:
                    bd = binary_dilation(hm, structure=st) & (~hm)
                else:
                    bd = _binary_dilation(hm) & (~hm)

                touch2 = int(np.count_nonzero(bd & (arr == 2)))
                touch3 = int(np.count_nonzero(bd & (arr == 3)))

                if touch2 == 0 and touch3 == 0:
                    # fallback: choose the closer fold by EDT
                    d2 = float(np.min(dist2[hm])) if np.isfinite(dist2).any() else np.inf
                    d3 = float(np.min(dist3[hm])) if np.isfinite(dist3).any() else np.inf
                    to_label = 2 if d2 <= d3 else 3
                else:
                    # choose the fold with stronger boundary contact (ties -> 2)
                    to_label = 2 if touch2 >= touch3 else 3

                arr[hm] = to_label  # never write 1 here (only background holes are filled)

        return arr

    mask_paths = glob(os.path.join(root_dir, "**", "*_mask.png"), recursive=True)
    print(f"🛠  Fixing {len(mask_paths)} masks under: {root_dir}")

    changed = 0
    for mp in mask_paths:
        try:
            lab = np.array(Image.open(mp).convert("L"), dtype=np.uint8)
            fixed = _process_single(lab)
            if not np.array_equal(lab, fixed):
                Image.fromarray(fixed, mode="L").save(mp)
                changed += 1
        except Exception as e:
            print(f"❌ Failed on {mp}: {e}")

    print(f"✅ Done. Updated {changed}/{len(mask_paths)} masks.")

In [ ]:
def copy_nth_of_each_hundred(root_dir, target_dir, n=50, copy_mask=True, overwrite=False):
    """
    From a flat folder `root_dir` containing files like '10001_rgb.png' (and usually
    '10001_mask.png'), copy only those frames whose index is the n-th within each
    100-block (e.g., n=50 → 10050, 10150, 10250, ...). Files are copied to `target_dir`
    keeping the original filenames.

    Args:
        root_dir   : source folder with *_rgb.png (and *_mask.png)
        target_dir : destination folder (created if missing)
        n          : which index inside each 100 (0..99); default 50
        copy_mask  : also copy the corresponding *_mask.png if present
        overwrite  : if False and a file exists in target, it is skipped

    Returns:
        List of integer indices copied (e.g., [10050, 10150, ...]).
    """
    import os, re, shutil

    if not (0 <= int(n) <= 99):
        raise ValueError("n must be in [0, 99]")

    os.makedirs(target_dir, exist_ok=True)
    want = n % 100

    rgb_regex = re.compile(r"^(\d+)_rgb\.png$", re.IGNORECASE)

    # gather candidate *_rgb.png files and filter by modulo rule
    candidates = []
    for name in os.listdir(root_dir):
        m = rgb_regex.match(name)
        if not m:
            continue
        idx = int(m.group(1))
        if idx % 100 == want:
            candidates.append((idx, name))

    # sort by numeric index
    candidates.sort(key=lambda x: x[0])

    copied = []
    for idx, rgb_name in candidates:
        src_rgb = os.path.join(root_dir, rgb_name)
        dst_rgb = os.path.join(target_dir, rgb_name)

        # copy RGB (respect overwrite flag)
        if overwrite or not os.path.exists(dst_rgb):
            shutil.copy2(src_rgb, dst_rgb)

        if copy_mask:
            stem = rgb_name.replace("_rgb.png", "")
            # prefer *_mask.png; fall back to *_seg4.png or *_seg.png if needed
            mask_candidates = [
                f"{stem}_mask.png",
                f"{stem}_seg4.png",
                f"{stem}_seg.png",
            ]
            for mn in mask_candidates:
                src_m = os.path.join(root_dir, mn)
                if os.path.exists(src_m):
                    dst_m = os.path.join(target_dir, mn)
                    if overwrite or not os.path.exists(dst_m):
                        shutil.copy2(src_m, dst_m)
                    break  # stop at first existing mask variant

        copied.append(idx)

    print(f"✅ Copied {len(copied)} frames to '{target_dir}'. Example indices: {copied[:5]}{'...' if len(copied)>5 else ''}")
    return copied

In [ ]:
source = r"C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension"
destination = r"C:\Users\olegp\miniconda3\envs\proj1\0_jupyter_notebook\data\train_extension\1"

copy_nth_of_each_hundred(source, destination)